# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# John Hopkins (Hoppy)
# Python API Homework
# GEGT Data Science Bootcamp
# 3/17/2021

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [76]:
city_vac_df = pd.read_csv("Clean_City_Data.csv", encoding = "utf-8") 

Unnamed: 0            696
City                  696
Max Temperature(F)    696
Humidity(%)           696
Cloudiness(%)         696
Wind Speed(mph)       696
Latitude              696
Longitude             696
Date                  696
dtype: int64

In [80]:
not_raining_clean_city_vac_df = city_vac_df.loc[(city_vac_df["Humidity(%)"] < 100)]
city_vac_weather_df = city_vac_df.dropna(how ="any")

In [81]:

pd.isna(city_vac_weather_df["Latitude"])

0      False
1      False
2      False
3      False
4      False
       ...  
691    False
692    False
693    False
694    False
695    False
Name: Latitude, Length: 696, dtype: bool

In [82]:
city_vac_weather_df["Latitude"]

0      62.7167
1     -23.7000
2      19.4000
3     -46.1927
4      72.7868
        ...   
691   -17.5333
692   -34.6534
693   -31.4241
694    41.2547
695    38.6979
Name: Latitude, Length: 696, dtype: float64

In [83]:
city_vac_weather_df.isna()

,Unnamed: 0,City,Max Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Latitude,Longitude,Date
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
691,False,False,False,False,False,False,False,False,False
692,False,False,False,False,False,False,False,False,False
693,False,False,False,False,False,False,False,False,False
694,False,False,False,False,False,False,False,False,False


In [85]:
city_vac_weather_df.count()

Unnamed: 0            696
City                  696
Max Temperature(F)    696
Humidity(%)           696
Cloudiness(%)         696
Wind Speed(mph)       696
Latitude              696
Longitude             696
Date                  696
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)


In [121]:
# Humidity Heatmap
location = city_vac_weather_df[["Latitude", "Longitude"]].astype(float)
humidity = city_vac_weather_df["Humidity(%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure(map_type = "HYBRID", zoom_level = 2,center = (15, 25))

# Create heat layer
heat_map_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating = False, point_radius = 5)

# Add layer
fig.max_intensity = max
fig.add_layer(heat_map_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [179]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows with null values
custom_city_vac_weather_df = city_vac_weather_df

custom_city_vac_weather_df = custom_city_vac_weather_df.loc[(custom_city_vac_weather_df["Max Temperature(F)"] < 80) & (custom_city_vac_weather_df["Max Temperature(F)"] > 70)]
custom_city_vac_weather_df = custom_city_vac_weather_df.loc[custom_city_vac_weather_df["Wind Speed(mph)"] < 10]
custom_city_vac_weather_df = custom_city_vac_weather_df.loc[custom_city_vac_weather_df["Cloudiness(%)"] == 20]
custom_city_vac_weather_df = custom_city_vac_weather_df.loc[(custom_city_vac_weather_df["Humidity(%)"] < 90) & (custom_city_vac_weather_df["Humidity(%)"] > 60)]
custom_city_vac_weather_df = custom_city_vac_weather_df.dropna()
custom_city_vac_weather_df

,Unnamed: 0,City,Max Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Latitude,Longitude,Date
14,16,anloga,78.80,78,20,6.91,5.7947,0.8973,1.616095e+09
38,41,teluknaga,79.00,83,20,3.44,-6.0989,106.6381,1.616095e+09
77,84,kattivakkam,77.00,73,20,5.01,13.2167,80.3167,1.616095e+09
179,197,souillac,78.80,73,20,3.44,-20.5167,57.5167,1.616095e+09
187,206,bambous virieux,78.80,73,20,3.44,-20.3428,57.7575,1.616095e+09
354,396,mahebourg,78.80,73,20,3.44,-20.4081,57.7000,1.616095e+09
398,443,quatre cocos,78.80,73,20,3.44,-20.2078,57.7625,1.616095e+09
447,494,kota bahru,78.01,83,20,3.51,6.1333,102.2386,1.616096e+09
503,553,ambon,77.00,88,20,8.05,-3.6954,128.1814,1.616096e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [180]:
hotel_data_df = custom_city_vac_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_data_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_data_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 14: anloga.
Closest hotel is Pin Drop Hotel.
------------
Retrieving Results for Index 38: teluknaga.
Closest hotel is Bale Ocasa hotel.
------------
Retrieving Results for Index 77: kattivakkam.
Closest hotel is Raj's villa.
------------
Retrieving Results for Index 179: souillac.
Closest hotel is Shanti Maurice Resort & Spa.
------------
Retrieving Results for Index 187: bambous virieux.
Closest hotel is Casa Tia Villa.
------------
Retrieving Results for Index 354: mahebourg.
Closest hotel is Shandrani Beachcomber Resort & Spa.
------------
Retrieving Results for Index 398: quatre cocos.
Closest hotel is Emeraude Beach Attitude Hotel.
------------
Retrieving Results for Index 447: kota bahru.
Closest hotel is Grand Riverview Hotel.
------------
Retrieving Results for Index 503: ambon.
Closest hotel is Swiss-Belhotel Ambon.
------------


In [181]:
hotel_data_df

,Unnamed: 0,City,Max Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Latitude,Longitude,Date,Hotel Name
14,16,anloga,78.80,78,20,6.91,5.7947,0.8973,1.616095e+09,Pin Drop Hotel
38,41,teluknaga,79.00,83,20,3.44,-6.0989,106.6381,1.616095e+09,Bale Ocasa hotel
77,84,kattivakkam,77.00,73,20,5.01,13.2167,80.3167,1.616095e+09,Raj's villa
179,197,souillac,78.80,73,20,3.44,-20.5167,57.5167,1.616095e+09,Shanti Maurice Resort & Spa
187,206,bambous virieux,78.80,73,20,3.44,-20.3428,57.7575,1.616095e+09,Casa Tia Villa
354,396,mahebourg,78.80,73,20,3.44,-20.4081,57.7000,1.616095e+09,Shandrani Beachcomber Resort & Spa
398,443,quatre cocos,78.80,73,20,3.44,-20.2078,57.7625,1.616095e+09,Emeraude Beach Attitude Hotel
447,494,kota bahru,78.01,83,20,3.51,6.1333,102.2386,1.616096e+09,Grand Riverview Hotel
503,553,ambon,77.00,88,20,8.05,-3.6954,128.1814,1.616096e+09,Swiss-Belhotel Ambon


In [182]:
hotel_df = hotel_data_df.dropna(how = "any")
hotel_df

,Unnamed: 0,City,Max Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Latitude,Longitude,Date,Hotel Name
14,16,anloga,78.80,78,20,6.91,5.7947,0.8973,1.616095e+09,Pin Drop Hotel
38,41,teluknaga,79.00,83,20,3.44,-6.0989,106.6381,1.616095e+09,Bale Ocasa hotel
77,84,kattivakkam,77.00,73,20,5.01,13.2167,80.3167,1.616095e+09,Raj's villa
179,197,souillac,78.80,73,20,3.44,-20.5167,57.5167,1.616095e+09,Shanti Maurice Resort & Spa
187,206,bambous virieux,78.80,73,20,3.44,-20.3428,57.7575,1.616095e+09,Casa Tia Villa
354,396,mahebourg,78.80,73,20,3.44,-20.4081,57.7000,1.616095e+09,Shandrani Beachcomber Resort & Spa
398,443,quatre cocos,78.80,73,20,3.44,-20.2078,57.7625,1.616095e+09,Emeraude Beach Attitude Hotel
447,494,kota bahru,78.01,83,20,3.51,6.1333,102.2386,1.616096e+09,Grand Riverview Hotel
503,553,ambon,77.00,88,20,8.05,-3.6954,128.1814,1.616096e+09,Swiss-Belhotel Ambon


In [183]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [184]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.5)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))